In [15]:
%pip install python-dotenv pandas chardet

Note: you may need to restart the kernel to use updated packages.


In [16]:
import_file = "George-CSV-Export/2023-09-01_2024-09-18.csv"

In [17]:
import chardet

# Open the CSV file in binary mode
with open(import_file, 'rb') as file:
    raw_data = file.read()

# Detect the encoding
result = chardet.detect(raw_data)

csv_encoding = result['encoding']

# Print the detected encoding
print(f"Detected encoding: {csv_encoding}")

Detected encoding: UTF-16


In [26]:
import pandas as pd

# Load CSV file into DataFrame
df = pd.read_csv(import_file, encoding=csv_encoding, parse_dates=["Buchungsdatum", "Valutadatum"], dayfirst=True)


# apply data type fixes
df['Highlight'] = df['Highlight'].astype(bool)
df['Betrag'] = pd.to_numeric(df['Betrag'], errors='coerce')

# Display the first few rows of the DataFrame
df.head()

,Eigener Kontoname,Eigene IBAN,Buchungsdatum,Partnername,Partner IBAN,BIC/SWIFT,Partner Kontonummer,Bankleitzahl,Betrag,Währung,...,Buchungsreferenz,Notiz,Highlight,Valutadatum,Virtuelle Kartennummer,Bezahlt mit,App,Zahlungsreferenz,Mandats ID,Creditor ID
0,Erneuerbare-Energie- Gemeinschaft I,AT112031405500057061,2024-09-17,NaN,NaN,NaN,NaN,NaN,NaN,EUR,...,203142409172CTG-0021T0JB5VK3,#eegfaktura,False,2024-09-17,NaN,NaN,NaN,NaN,NaN,AT07ZZZ00000079253
1,Erneuerbare-Energie- Gemeinschaft I,AT112031405500057061,2024-09-16,NaN,NaN,NaN,NaN,NaN,NaN,EUR,...,203142409162CTG-00A3RPMD7DG1,#eegfaktura,False,2024-09-16,NaN,NaN,NaN,NaN,NaN,NaN
2,Erneuerbare-Energie- Gemeinschaft I,AT112031405500057061,2024-09-10,Sparkasse Salzkammergut,AT682031491718500118,SKBIAT21XXX,9.171850e+10,20314.0,NaN,EUR,...,203142409102AEI-F99BB0WA23MC,#sponsoring,False,2024-09-10,NaN,NaN,NaN,NaN,NaN,NaN
3,Erneuerbare-Energie- Gemeinschaft I,AT112031405500057061,2024-09-05,Google Cloud EMEA Limited,IE35CITI99005132549640,CITIIE2XXXX,NaN,NaN,NaN,EUR,...,201002409022AEI-40XQH8000954,#förderbar,False,2024-09-05,NaN,NaN,NaN,NaN,51126908,IE34ZZZ362297
4,Erneuerbare-Energie- Gemeinschaft I,AT112031405500057061,2024-08-16,"Kulturverein ""einundzwanzig""",AT871500000161213418,OBKLAT2LXXX,NaN,15000.0,NaN,EUR,...,201002408162AEI-27XVU9000856,#honorar,False,2024-08-16,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df.columns

Index(['Eigener Kontoname', 'Eigene IBAN', 'Buchungsdatum', 'Partnername',
       'Partner IBAN', 'BIC/SWIFT', 'Partner Kontonummer', 'Bankleitzahl',
       'Betrag', 'Währung', 'Buchungs-Details', 'Buchungsreferenz', 'Notiz',
       'Highlight', 'Valutadatum', 'Virtuelle Kartennummer', 'Bezahlt mit',
       'App', 'Zahlungsreferenz', 'Mandats ID', 'Creditor ID'],
      dtype='object')

In [28]:


import psycopg
import os
import pytz

os.environ['PGSERVICEFILE'] = "/home/martin/Workspace/Energiegemeinschaft/notebooks/.pg_service.conf"
os.environ['PGPASSFILE'] = "/home/martin/Workspace/Energiegemeinschaft/notebooks/.pgpass"

with psycopg.connect(service='eeg-middleware') as conn:
    with conn.cursor() as cur:
        
        
        sql = """
        INSERT INTO accounting_booking (
            account_name,
            iban,
            booking_date,
            partner_name,
            partner_iban,
            bic_swift,
            partner_account_number,
            bank_code,
            amount,
            currency,
            booking_details,
            booking_reference,  -- unique reference
            note,
            highlight,
            value_date,
            virtual_card_number,
            paid_with,
            app,
            payment_reference,
            mandate_id,
            creditor_id
        )
        VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        ON CONFLICT (booking_date, partner_iban, amount, currency, booking_details, value_date, mandate_id, creditor_id, payment_reference, booking_reference)
        DO UPDATE SET
            account_name = EXCLUDED.account_name,
            iban = EXCLUDED.iban,
            booking_date = EXCLUDED.booking_date,
            partner_name = EXCLUDED.partner_name,
            partner_iban = EXCLUDED.partner_iban,
            bic_swift = EXCLUDED.bic_swift,
            partner_account_number = EXCLUDED.partner_account_number,
            bank_code = EXCLUDED.bank_code,
            amount = EXCLUDED.amount,
            currency = EXCLUDED.currency,
            booking_details = EXCLUDED.booking_details,
            note = EXCLUDED.note,
            highlight = EXCLUDED.highlight,
            value_date = EXCLUDED.value_date,
            virtual_card_number = EXCLUDED.virtual_card_number,
            paid_with = EXCLUDED.paid_with,
            app = EXCLUDED.app,
            payment_reference = EXCLUDED.payment_reference,
            mandate_id = EXCLUDED.mandate_id,
            creditor_id = EXCLUDED.creditor_id;
        """


        for index, row in df.iterrows():
            values = (
                row['Eigener Kontoname'],  # account_name
                row['Eigene IBAN'],  # iban
                row['Buchungsdatum'],  # booking_date
                row['Partnername'],  # partner_name
                row['Partner IBAN'],  # partner_iban
                row['BIC/SWIFT'],  # bic_swift
                row['Partner Kontonummer'],  # partner_account_number
                row['Bankleitzahl'],  # bank_code
                row['Betrag'],  # amount
                row['Währung'],  # currency
                row['Buchungs-Details'],  # booking_details
                row['Buchungsreferenz'],  # booking_reference (unique key)
                row['Notiz'],  # note
                row['Highlight'],  # highlight
                row['Valutadatum'],  # value_date
                row['Virtuelle Kartennummer'],  # virtual_card_number
                row['Bezahlt mit'],  # paid_with
                row['App'],  # app
                row['Zahlungsreferenz'],  # payment_reference
                row['Mandats ID'],  # mandate_id
                row['Creditor ID']  # creditor_id
            )
            # Execute the SQL query for each row
            cur.execute(sql, values)
        
        conn.commit()


